In [142]:
import os
import cv2
import numpy as np
import random
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from collections import OrderedDict
import re 

In [171]:
class newModel:

    def __init__(self, target_x, target_y, folder, model, resize_function, chunk_size=100):
        self.target_x = target_x
        self.target_y = target_y
        self.folder = folder
        self.model = model
        self.resize_function = resize_function
        self.chunk_size = chunk_size
        self.labels = []

    def apply_features(self, img):
        '''Apply different features such as: Edge Filters, garbor filters, etc...'''
        return img

    def resize(self, img):
        ''' Output will change based on resize function (Bilinear, Randomized Padding, etc.)'''
        return img
    
    def augment(self, img):
        return img
    
    def gen_labels(self, files):
        for filename in files:
            pattern = r'^(.*?)_(\d+)\.jpg$'
            match = re.match(pattern, filename)
            if match:
                name_without_number = match.group(1)
            else:
                print("Error with ", filename)

            # Albrecht Check
            if name_without_number == 'Albrecht_Du╠êrer':
                continue
        
            self.labels.append(name_without_number)

        self.label_mapping = OrderedDict((label, index) for index, label in enumerate(set(self.labels)))
        self.categorical_values = to_categorical(list(self.label_mapping.values()))
        self.categorical_map = {value[0]: self.categorical_values[i] for i, value in enumerate(self.label_mapping.items())}
        self.test_categorical = {str(value): key for key, value in self.categorical_map.items()}
        

        # Convert string labels to integer labels
        self.integer_labels = [self.categorical_map[label] for label in self.labels]
        
        self.labels = self.integer_labels
    
    def train(self, images, labels):
        images = images.astype('float32') / 255.0
        self.model.fit(images, labels)

    def load_images_and_train(self):
        '''Function to load in images from a folder.'''
        images = []
        labels = []
        cur_index = 0
        self.files = os.listdir(self.folder)
        random.shuffle(self.files)

        self.gen_labels(self.files)
        stopping_point = len(self.labels)
        
        for filename in self.files:
            
            if cur_index == stopping_point-1:
                # Train current array and labels
                self.train(np.array(images), np.array(labels))
                
                # Empty out variables
                images, labels = [], []

            if 'Albrecht_Du╠êrer' in filename:
                continue

            if filename.endswith('.jpg'):
                
                img_path = os.path.join(self.folder, filename)
                img = load_img(img_path, target_size=(self.target_x, self.target_y))  # Resize images if necessary

                img = self.resize(img) # use the current resize function for this iteration.

                img = self.augment(img) # Create different augmentations of the image.

                img = self.apply_features(img) # TODO: Make this a list of images or image arrays

                img_array = img_to_array(img) # TODO: Convert list of images to multi-dimension array of images.
                images.append(img_array) # Append Image with its filters to images 
                 
                labels.append(self.labels[cur_index])
                
                cur_index += 1    
            
            if (cur_index % self.chunk_size) == 0:
                # Train current array and labels

                print(np.array(images).shape, np.array(labels).shape)
                self.train(np.array(images), np.array(labels))
                
                # Empty out variables
                images, labels = [], []
            
            
            
                

In [172]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(240, 240, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(50, activation='softmax'))  # Replace `num_classes` with the actual number of classes
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [173]:
folder = '/home/ceg98/Documents/archive/resized/resized/'
nm = newModel(240, 240, folder, model, None, 100)

In [174]:
nm.load_images_and_train()

(100, 240, 240, 3) (100, 50)
4/4 [==============================] - 1s 10ms/step - loss: 44.9480 - accuracy: 0.0200
(100, 240, 240, 3) (100, 50)
4/4 [==============================] - 0s 10ms/step - loss: 28.6044 - accuracy: 0.0200
(100, 240, 240, 3) (100, 50)
4/4 [==============================] - 0s 10ms/step - loss: 11.1932 - accuracy: 0.0100
(100, 240, 240, 3) (100, 50)
4/4 [==============================] - 0s 10ms/step - loss: 5.7660 - accuracy: 0.0400
(100, 240, 240, 3) (100, 50)
4/4 [==============================] - 0s 10ms/step - loss: 4.1849 - accuracy: 0.0100
(100, 240, 240, 3) (100, 50)
4/4 [==============================] - 0s 11ms/step - loss: 3.9319 - accuracy: 0.0300
(100, 240, 240, 3) (100, 50)
4/4 [==============================] - 0s 11ms/step - loss: 3.9014 - accuracy: 0.0300
(100, 240, 240, 3) (100, 50)
4/4 [==============================] - 0s 10ms/step - loss: 3.8911 - accuracy: 0.0500
(100, 240, 240, 3) (100, 50)
4/4 [==============================] - 0s 10ms/s